# Cours : Tables de hachage

Dans ce cours, on présente une nouvelle structure de données : *les tables de hachage*. Les tables de hachage servent en particulier à stocker un ensemble de valeurs (sans qu'elles soient ordonnées) en permettant de  : 
- savoir rapidement si une valeur est ou non dans la table, 
- ajouter/supprimer rapidement des valeurs dans la table.

Une structure de données permettant de déterminer rapidement si une valeur appartient à un ensemble est la liste triée (recherche dichotomique). En revanche, ajouter/supprimer des valeurs n'est pas efficace car il faut maintenir la liste triée.

Une liste non triée (ne prenant pas en compte un ordre sur les éléments) permet d'ajouter/supprimer rapidement des éléments mais la recherche d'un élément dans la liste n'est pas rapide (complexité linéaire).

Les tables de hachage permettent (sous certaines conditions) d'avoir les deux propriétés simultanément.

**Remarque :** Les dictionnaires en python sont des tables de hachage (une version plus complexe que les tables de hachage présentées dans ce chapitre).

## Principe

On souhaite stocker l'ensemble de nombres suivant : 

12, 45, 34, 67, 78, 345, 7069, 59482, 4, 2, 34534, 57, 3321, 23, 543, -12, 847, -342, 897, 0, -1

Si l'on stocke cet ensemble sous la forme d'un tableau :

In [ ]:
tab = [12, 45, 34, 67, 78, 345, 7069, 59482, 4, 2, 34534, 57, 3321, 23, 543, 18, 847, 342, 897, 0, 1]

alors tester si un nombre est dans mon tableau nécessite de tester 21 valeurs dans le pire des cas.

Si l'on sépare les nombres pairs et impairs en deux tableaux distincts : 

In [ ]:
pairs = [12, 34, 78, 59482, 4, 2, 34534, 18, 342, 0]
impairs = [ 45, 67, 345, 7069, 57, 3321, 23, 543, 847, 897, 1]

tester si une valeur est l'ensemble nécessitera alors au plus 10 tests si la valeur est paire, et 11 tests si la valeur est impaire.

On peut même faire mieux en séparant les nombres qui se terminent par 0, ceux qui se terminent par 1, etc.

In [ ]:
fini_par_0 = [0]
fini_par_1 = [3321, 1]
fini_par_2 = [12, 59482, 2, 342]
fini_par_3 = [23, 543]
fini_par_4 = [34, 4, 34534]
fini_par_5 = [45, 345]
fini_par_6 = []
fini_par_7 = [67, 57, 847, 897]
fini_par_8 = [78, 18]
fini_par_9 = [7069]

Dans ce cas, pour déterminer si 97047 appartient à l'ensemble, il suffit de chercher si ce nombre est dans la liste des nombres se terminant par 7. Il faut donc chercher parmi 4 valeurs (et non plus 21).
Dans cet exemple, la recherche d'un nombre nécessite  au plus 4 tests.

La recherche est plus rapide mais requiert 10 tableaux. Pour implanter efficacement cette stratégie, on utilise une structure de données imbriquée : on crée une variable `table` qui correspond à un tableau de 10 cases. Pour tout indice `i` (entre 0 et 9), `table[i]` est un tableau contenant l'ensemble des nombres se terminant par le chiffre `i`.

De cette manière, les 10 tableaux précédents sont stockés sous la forme : 

In [ ]:
table = [
    [0],
    [3321, 1],
    [12, 59482, 2, 342],
    [23, 543],
    [34, 4, 34534],
    [45, 345],
    [],
    [67, 57, 847, 897],
    [78, 18],
    [7069]
]

On peut alors coder des fonctions pour chercher si un entier apparaît dans la table, ajouter des entiers dans la table ou en supprimer.

**Remarque :** Quand l'entier est positif, on peut utiliser l'opérateur `entier % 10` pour déterminer le dernier chiffre et donc l'indice dans la table. Si l'entier est négatif, on obtient le dernier chiffre avec l'instruction `-entier % 10`.

## Fonctions de hachage

Si l'exemple précédent est efficace quand on a une petite liste des nombres, il pose cependant quelques problèmes :
- si le nombre de valeurs à stocker est très grand (par exemple 100 000), alors chaque case de la table va contenir en moyenne 10 000 entiers. La recherche d'une valeur nécessitera jusqu'à 10 000 comparaisons ce qui n'est pas efficace. Pour améliorer la recherche, il faudrait pouvoir faire la même chose avec une table ne contenant plus 10 cases mais 10 000 voir 100 000 cases, et faire en sorte que chacune des cases ne contienne que très peu d'entiers ;
- si l'on doit stocker uniquement des multiples de 10, alors tous les nombres seront dans `table[0]`, les autres cases ne contenant alors que des tableaux vides. La table est aussi peu performante pour la recherche qu'un tableau non trié ; 
- si l'on veut stocker des valeurs non entières, il faut procéder autrement pour séparer les valeurs.

L'utilisation des fonctions de hachage permet de pallier ces différents problèmes.

### Principe d'une fonction de hachage

Une *fonction de hachage* est une fonction prenant en paramètre une valeur et lui associant un entier. Elle doit respecter les conditions suivantes :
- être rapide,
- produire le même résultat lorsqu'elle est appelée avec la même valeur.

Pour n'importe quelle valeur, l'entier retourné par la fonction de hachage **modulo la taille de la table** correspond alors à l'indice de la case du tableau dans laquelle la valeur doit être stockée.

Par exemple, pour créer une table de hachage contenant des chaînes de caractères, il faut concevoir une fonction de hachage qui associe à chaque chaîne de caractères un entier. On peut par exemple choisir d'additionner les codes ASCII des caractères formant la chaîne : 

In [1]:
def hachage(str):
    nombre = 0
    i = 0
    while i < len(str):
        nombre += ord(str[i])
        i += 1
    return nombre


print("hachage de a : ", hachage("a"))
print("hachage de b : ", hachage("b"))
print("hachage de bonjour : ", hachage("bonjour"))
print("hachage de ab : ", hachage("ab"))
print("hachage de ba : ", hachage("ba"))

hachage de a :  97
hachage de b :  98
hachage de bonjour :  767
hachage de ab :  195
hachage de ba :  195


Grâce à la fonction de hachage, on peut déterminer dans quelle case de la table chercher : l'indice de la chaîne `str` sera donné par `hachage(str)%len(table)`.

## Notions de performances

L'efficacité de la recherche, de l'ajout et de la suppression dépend grandement du nombre de valeurs contenues dans une même case de la table. Si l'on note $c$ le nombre maximal de valeurs dans une même case de la table, alors la recherche, l'ajout et la suppression se font en $O(c)$.

Idéalement, on cherche à avoir une table de hachage très grande avec des valeurs bien réparties dans les différentes cases de manière à avoir une valeur petite pour $c$.

**Remarques :**

* La valeur $c$ dépend directement de la fonction de hachage. Le choix de la fonction de hachage a donc une grande influence sur l'efficacité de la table de hachage. 

  Une fonction de hachage *parfaite* répartit uniformément les différentes valeurs dans les cases de la table (même nombre de valeurs dans chaque case de la table).

* La valeur de $c$ varie au cours des ajouts/suppressions. Au départ, quand la table est vide, elle vaut 0. Dans le pire des cas, elle est égale au nombre de valeurs dans la table (si toutes les valeurs sont rangées dans la même case).

* La dimension de la table est importante pour l'efficacité. Même si la fonction de hachage est *parfaite*, $c \geq \frac{n}{k}$ où $n$ est le nombre de valeurs dans la table et $k$ le nombre de cases de la table.

  Il faut donc avoir une table avec suffisamment de cases par rapport au nombre de valeurs à stocker, quitte à recopier une table de hachage dans une autre plus grande.